## Guidelines

This intended use of this notebook is as an *independent* source for results used in AEIC test cases. As such, no code may be imported from AEIC modules here. Ideally, only Python standard library imports should be used.

For each test case, there should be an **Independent implementation** section (with an implementation of whatever is being tested that is as independent as possible from the code in AEIC), a **Test data** section and a **Test case evaluation** section. The idea is that the contents of the **Test data** and **Test case evaluation** sections can be copied and pasted into a real Python test function but that the implementation getting from one to the other is independent of what happens in the AEIC code.

(The word "independent" here is doing a lot of work. In an ideal world, the independent implementation would be written by someone other than the person writing the implementation in AEIC, without too much communication other than "I'm implementing equation (X) from this paper", but that's obviously impractical in many cases. If you find yourself needing to implement both the test case generation and the actual AEIC code, try to split your mind and keep them independent!)

In [1]:
import math

### 1. Sea-level-static fuel flow

The `get_SLS_equivalent_fuel_flow` function converts fuel flow at altitude to its equivalent at sea level using calculations from the Boeing Fuel Flow Method 2 (BFFM2) from DuBois, D. & Paynter, G. (2006). *Fuel Flow Method 2 for Estimating Aircraft Emissions*. SAE Technical Paper 2006-01-1987.

Here we generate independent test data using an implementation of the method taken directly from equation (40) in the paper.

#### Independent implementation

Define some constants:

In [2]:
# Sea level pressure [Pa].
pressure_SL = 101325.0

# Sea level temperature [K].
temperature_SL = 288.15

# Ratio of specific heats at constant pressure and constant volume for dry air.
kappa = 1.4

# Specific gas constant for dry air [J kg-1 K-1].
R_air = 287.05287

Define a function to calculate equation (40) from the paper (we need fuel flow, pressure, temperature and airspeed, since the equation makes use of the Mach number):

In [3]:
def fuel_flow_to_sls(fuel_flow: float, pressure: float, temperature: float, air_speed: float) -> float:
    # Mach number.
    M = air_speed / math.sqrt(kappa * R_air * temperature)

    # Non-dimensional pressure and temperature.
    delta_amb = pressure / pressure_SL
    theta_amb = temperature / temperature_SL

    # Equation (40) from paper.
    # print(f'M = {M}  delta_amb = {delta_amb}  theta_amb = {theta_amb}')
    return fuel_flow * theta_amb**3.8 / delta_amb * math.exp(0.2 * M**2)

#### Test data

Some of this test data is generated from the `AtmosphericState` class in the AEIC code.

In [4]:
# Reference altitudes [m] along test trajectory.
altitudes = [0.0, 1500.0, 6000.0, 11000.0, 9000.0, 2000.0]

# Temperatures [K] and pressures [Pa] along test trajectory (calculated from ISA atmosphere).
temperatures = [288.15, 278.4, 249.15, 216.65, 229.65, 275.15]
pressures = [101325.0, 84555.9940737564, 47181.0021852292, 22632.0400950078, 30742.4326120969, 79495.201934051]

# Total fuel flows along test trajectory [kg/s].
full_fuel_flows = [0.3, 0.35, 0.55, 0.65, 0.5, 0.32]

# Per-engine fuel flows along test trajectory [kg/s].
number_of_engines = 2
fuel_flows = [ff / number_of_engines for ff in full_fuel_flows]

# Airspeed along test trajectory [m/s].
airspeeds = [120.0, 150.0, 190.0, 210.0, 180.0, 140.0]

#### Evaluate test case

In [5]:
results = list(map(fuel_flow_to_sls, fuel_flows, pressures, temperatures, airspeeds))

In [6]:
results

[0.15377734749955685,
 0.19154479109277428,
 0.3652574468094632,
 0.5447580164850215,
 0.37317567328909007,
 0.17729854838504117]

Rounded results for use in test:

In [7]:
[round(r, 6) for r in results]

[0.153777, 0.191545, 0.365257, 0.544758, 0.373176, 0.177299]